In [1]:
# Random Imports
import os
import sys
import yaml
from pathlib import Path
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSeg')

# Regular schema dictates that we put DATAPATH
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
))
os.environ['WANDB_NOTEBOOK_NAME'] = 'train.ipynb'

from ionpy.util import Config
from ionpy.util.config import check_missing
from ese.scripts.utils import get_option_product

from datetime import datetime
# Get today's date
today_date = datetime.now()
# Format the date as MM_DD_YY
formatted_date = today_date.strftime("%m_%d_%y")

# Setup direcrtories
root = Path("/storage/vbutoi/scratch/ESE")
# Define some useful paths.
scratch_root = Path("/storage/vbutoi/scratch/ESE")
code_root = Path("/storage/vbutoi/projects/ESE")

%load_ext yamlmagic
%load_ext autoreload
%autoreload 2

## Define said jobs

In [2]:
%%yaml default_cfg 

experiment:
  seed: '?' 
    
dataloader:
  batch_size: '?' 
  num_workers: '?' 
  pin_memory: True 

optim: 
  _class: torch.optim.Adam
  lr: 3.0e-4
  weight_decay: 0.0 
  
train:
  epochs: 1000 
  eval_freq: 5 

log:
  checkpoint_freq: 20 
  root: '?'
  metrics:
    dice_score:
      _fn: ionpy.metrics.dice_score
      from_logits: True
      batch_reduction: 'mean' 
      ignore_empty_labels: True # False for WMH, True for everything else. 
      # ignore_index: 0 # Ignore background class when reporting.
    ece_loss:
      _fn: ese.experiment.metrics.image_ece_loss
      from_logits: True
      num_bins: 15
      # ignore_index: 0 Useful for binary
    edge_ece_loss:
      _fn: ese.experiment.metrics.image_edge_ece_loss
      from_logits: True
      num_bins: 15
      neighborhood_width: 3
      # ignore_index: 0 Useful for binary
    elm_loss:
      _fn: ese.experiment.metrics.image_elm_loss
      from_logits: True
      num_bins: 15
      neighborhood_width: 3
      # ignore_index: 0 Useful for binary

######################
# Cross-Entropy Loss #
######################
# loss_func: 
#   _class: ionpy.loss.PixelCELoss
#   from_logits: True
#   batch_reduction: 'mean' 
  
#############
# Dice Loss #
#############
loss_func: 
  _class: ionpy.loss.SoftDiceLoss
  from_logits: True
  batch_reduction: 'mean' 
  ignore_empty_labels: True # False, for WMH, True for everything else.
  # ignore_index: 0 

<IPython.core.display.Javascript object>

In [3]:
%%yaml model_cfg  

model:
  _class: ese.experiment.models.unet.UNet
  filters: [128, 128, 128, 128, 128]
  convs_per_block: 3

<IPython.core.display.Javascript object>

In [4]:
%%yaml callbacks_cfg

callbacks:
  step:
    - ese.experiment.callbacks.ShowPredictions
  epoch:
    - ese.experiment.callbacks.WandbLogger
    - ionpy.callbacks.ETA
    - ionpy.callbacks.JobProgress
    - ionpy.callbacks.TerminateOnNaN
    - ionpy.callbacks.PrintLogged
    - ionpy.callbacks.ModelCheckpoint:
        monitor: dice_score
        phase: val

<IPython.core.display.Javascript object>

In [5]:
%%yaml aug_cfg 

# AUGMNENTATIONS USED FOR CITYSCAPES
augmentations:
    train:
        - Resize:
            height: 256
            width: 512
        - HorizontalFlip:
            p: 0.5
    val:
        - Resize: # Maybe strange to do this, but we want to be able to compare the results.
            height: 256
            width: 512

<IPython.core.display.Javascript object>

## Debug Station

In [6]:
# Setup the root.
exp_name = '02_22_24_Shapes_Pilot'
# add_string = '_HippocampusTraining'
# exp_root = str(root / 'training' / (formatted_date + add_string) / exp_name)
exp_root = str(root / 'training' / exp_name)

# Create the ablation options
option_set = [
    {
        'log.root': [exp_root],
        'data.preload': [False],
        'dataloader.batch_size': [4],
        'dataloader.num_workers': [2],
        'experiment.seed': [40, 41, 42, 43]
    }
]

In [7]:
# Load the inference cfg from local.
##################################################
cal_cfg_root = code_root / "ese" / "experiment" / "configs" / "training"
##################################################
with open(cal_cfg_root / "Shapes.yaml", 'r') as file:
    dataset_cfg = yaml.safe_load(file)

# Assemble base config
base_cfg = Config(default_cfg).update([model_cfg, dataset_cfg, callbacks_cfg])

# Get the configs
raw_cfgs = get_option_product(exp_name, option_set, base_cfg)

cfgs = []
for cfg in raw_cfgs:
    cfg = cfg.update(aug_cfg)
    check_missing(cfg) # Verify there are no ? in config.
    cfgs.append(cfg)

In [8]:
len(cfgs)

4

## Running Jobs

In [9]:
######## FOR DEBUGGING
from ese.experiment.experiment import run_ese_exp, CalibrationExperiment

run_ese_exp(
    config=cfgs[0], 
    experiment_class=CalibrationExperiment,
    gpu='3',
    run_name='debug',
    show_examples=False,
    track_wandb=False
)

/local/vbutoi/envs/UniverSegTF/lib/python3.9/site-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_outputs" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


Set seed: 40


/storage/vbutoi/projects/ionpy/util/libcheck.py:49: UserWarning: Intel MKL extensions not available for NumPy
  warn("Intel MKL extensions not available for NumPy")
/storage/vbutoi/projects/ionpy/util/libcheck.py:57: UserWarning: Using slow Pillow instead of Pillow-SIMD
  warn("Using slow Pillow instead of Pillow-SIMD")


ValidationError: 2 validation errors for Unet
in_channels
  Field required [type=missing, input_value={'convs_per_block': 3, 'f...28, 128, 128, 128, 128]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.5/v/missing
out_channels
  Field required [type=missing, input_value={'convs_per_block': 3, 'f...28, 128, 128, 128, 128]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.5/v/missing

In [ ]:
# # FOR SUBMISSION
# from ese.experiment.experiment import submit_ese_exps, CalibrationExperiment 

# submit_ese_exps(
#     config_list=cfgs,
#     experiment_class=CalibrationExperiment,
#     available_gpus=['0', '1', '2', '3'],
#     # available_gpus=['3', '4', '5', '6', '7'],
#     track_wandb=True
# )